In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt




class NeutrinoElectronScattering:

    neutrinoelectron = 12
    neutrinomuon = 14
    neutrinotau = 16

    def __init__(self):
        self.sin2w = 0.23121
        self.gv = -1. / 2. + 2. * self.sin2w
        self.ga = -1. / 2.
        self.me = 0.510998950  # MeV
        self.mmu = 105.6583755  # MeV
        self.mtau = 1776.86  # MeV

        self.Pi = 3.1415927
        self.GF = 1.166378e-5  # GeV^-2
        self.GF2 = self.GF * self.GF
        self.MeV2mbarn = 1. / 2.56819e+6

        print("Neutrino-Electron scattering initialized")

    def electronmass(self):
        return self.me

    def muonmass(self):
        return self.mmu

    def taumass(self):
        return self.mtau

    def S(self, Enu):
        return (Enu + self.electronmass()) ** 2 - Enu ** 2

    def Y(self, Enu, Ee):
        return (Ee - self.electronmass()) / Enu

    def Normalization(self):
        return self.GF2 / self.Pi * self.MeV2mbarn

    def diffcrosssection(self, Enu, El, neutrino):
        aneut = False

        if neutrino < 0:
            aneut = True

        xs = 0

        if El >= self.electronmass():
            if abs(neutrino) == self.neutrinoelectron:
                xs = self.nuee(Enu, El, aneut)
            else:
                xs = self.nule(Enu, El, aneut)

        if xs < 0:
            xs = 0

        return xs

    def test(self):

      return { "cross-section": 0, "El": 0, ... }

    def GENcrosssection(self, Enu, El, cos, neutrino):
        aneut = False

        if neutrino < 0:
            aneut = True

        xs = 0

        if El >= self.electronmass():
            if abs(neutrino) == self.neutrinoelectron:
                xs = self.GENnuee(Enu, El, cos, aneut)
            else:
                xs = self.GENnule(Enu, El, cos, aneut)

        if xs < 0:
            xs = 0

        return xs

    def Totalcrosssection(self, Enu, neutrino):
        aneut = False

        if neutrino < 0:
            aneut = True

        xs = 0

        if abs(neutrino) == self.neutrinoelectron:
            xs = self.nueeInt(Enu, aneut)
        else:
            xs = self.nuleInt(Enu, aneut)

        if xs < 0:
            xs = 0

        return xs

    def GetCosine(self, Enu, El, neutrino):
        mass = self.me

        Pl = math.sqrt(El ** 2 - mass ** 2)

        return (El * mass + Enu * El - mass ** 2 - Enu * mass) / (Enu * Pl)

    def GetEnu(self, Cosine, El, neutrino):
        mass = self.me

        Pl = math.sqrt(El ** 2 - mass ** 2)

        return (El * mass - mass ** 2) / (-El + mass + Pl * Cosine)

    def GenerateY(self, CLL, CLR, N):
        r = random.random()

        A = CLL ** 2 / N
        B = CLR ** 2 / N
        C = r

        a2 = 3. * A / B
        a3 = 3. * (C - B / 3. - A) / B

        Q = a2 / 3.
        R = -a3 / 2.

        discriminant = Q ** 3 + R ** 2

        S1 = (R + math.sqrt(discriminant)) ** (1. / 3.)
        S2 = -(math.sqrt(discriminant) - R) ** (1. / 3.)

        x1 = S1 + S2

        return 1 - x1

    def nule(self, Enu, Ee, aneut=False):
        y = self.Y(Enu, Ee)

        if not aneut:
            CLL = -1. / 2. + self.sin2w
            CLR = self.sin2w
        else:
            CLL = self.sin2w
            CLR = -1. / 2. + self.sin2w

        return self.Normalization() * self.S(Enu) * (CLL ** 2 + CLR ** 2 * (1 - y) ** 2)

    def nuleInt(self, Enu, aneut=False):
        norm = self.GF2 / self.Pi * self.MeV2mbarn
        s = self.S(Enu)

        CLL = -1. / 2. + self.sin2w
        CLR = self.sin2w

        if Enu == 0:
            ymax = 0
        else:
            ymax = (Enu - self.me) / Enu

        return norm * s * (CLL ** 2 * ymax + CLR ** 2 * 1. / 3. * (-(1 - ymax) ** 3 + 1))

    def GENnule(self, Enu, El, cos, aneut=False):
        norm = self.GF2 / self.Pi * self.MeV2mbarn
        s = self.S(Enu)

        CLL = -1. / 2. + self.sin2w
        CLR = self.sin2w

        if aneut:
            CLL = self.sin2w
            CLR = -1. / 2. + self.sin2w

        ymax = (Enu - self.me) / Enu

        N = self.nuleInt(Enu, aneut) / (norm * s)

        y1 = self.GenerateY(CLL, CLR, N)

        El = y1 * Enu + self.me

        cos = self.GetCosine(Enu, El, self.neutrinoelectron)

        return self.nule(Enu, El, aneut)

    def nueeInt(self, Enu, aneut=False):
        norm = self.GF2 / self.Pi * self.MeV2mbarn
        s = self.S(Enu)

        CLL = 1. / 2. + self.sin2w
        CLR = self.sin2w

        if Enu == 0:
            ymax = 0
        else:
            ymax = (Enu - self.me) / Enu

        return norm * s * (CLL ** 2 * ymax + CLR ** 2 * 1. / 3. * (-(1 - ymax) ** 3 + 1))

    def nuee(self, Enu, Ee, aneut=False):
        y = self.Y(Enu, Ee)

        if not aneut:
            CLL = 1. / 2. + self.sin2w
            CLR = self.sin2w
        else:
            CLL = -1. / 2. + self.sin2w
            CLR = 1. / 2. + self.sin2w

        return self.Normalization() * self.S(Enu) * (CLL ** 2 + CLR ** 2 * (1 - y) ** 2)

    def GENnuee(self, Enu, El, cos, aneut=False):
        CLL = 1. / 2. + self.sin2w
        CLR = self.sin2w

        ymax = (Enu - self.me) / Enu

        N = self.nueeInt(Enu, aneut) / self.Normalization() / self.S(Enu)

        y1 = self.GenerateY(CLL, CLR, N)

        El = y1 * Enu + self.me

        cos = self.GetCosine(Enu, El, self.neutrinoelectron)

        return self.nuee(Enu, El, aneut)


    def GENnuee_example(self, Enu, aneut = False):

      xs = float()
      El = float()
      cos = float()


      CLL = float()
      CLR = float()

      if aneut:
        CLL = -1./2.+self.sin2w;
        CLR = 1./2.+self.sin2w;
      else:
        CLL = 1./2.+self.sin2w;
        CLR = self.sin2w;

      ymax = (Enu-self.me)/Enu;

      N = nueeInt(Enu,aneut)/Normalization()/S(Enu);

      y1 = GenerateY(CLL,CLR,N);

      El = y1*Enu+self.me;

      cos = GetCosine(Enu,El,self.neutrinoelectron);

      return [xs, El, cos]


    def GENcrosssection_example(self, Enu, neutrino):
      aneut = False
      if neutrino < 0: aneut = True;  # It is anti-neutrino

      xs = float()
      El = float()
      cos = float()

      if abs(neutrino) == self.neutrinoelectron:
        out = self.GENnuee_example(Enu,aneut)
        xs = out[0]
        El = out[1]
        cos = out[2]
      # else:
      #   xs = GENnule(Enu,El,cos,aneut);

      if xs < 0: xs = 0

      return [ xs, El, cos ]


def crossSection(EnuIn = 10., Emin = 0):
  nul = NeutrinoElectronScattering()

  neutrinotype = nul.neutrinoelectron

  r = random.Random()
  if EnuIn < 0:
      Enu = r.uniform(Emin, -EnuIn)
  else:
      Enu = EnuIn

  cos = nul.GetCosine(Enu, Enu, neutrinotype)

  El = nul.GetEnu(cos, Enu, neutrinotype)

  xs = nul.GENcrosssection(Enu, El, cos, neutrinotype)

  print(f"Enu: {Enu}, El: {El}, cos: {cos}, Cross-section: {xs}")



crossSection()
